# X. Model Inference

#### Import Libraries

In [17]:
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import string
import ast
from tensorflow.keras.models import load_model


#### Import Functions

In [18]:
#open chatwords.txt
with open('chatwords.txt') as f:
    data = f.read()
chatwords =  ast.literal_eval(data)
chatwords

#open abbreviation.txt
with open('abbreviation.txt') as ab:
    ab2 = ab.read()
abbreviation =  ast.literal_eval(ab2)
abbreviation

#define stopwords
stop_words = stopwords.words('english')

#define lemmatizer
lem = WordNetLemmatizer()

In [19]:
def check_chatwords(text):
    temp=[]
    for chat in text.split():
        if chat.upper() in chatwords:
            temp.append(chatwords[chat.upper()])
        else:
            temp.append(chat)
    return " ".join(temp)

def lower(text):
    data = text.lower()
    return data 

def check_abbr(text):
    temp2=[]
    for abbr in text.split():
      if abbr in abbreviation:
          temp2.append(abbreviation[abbr])
      else:
          temp2.append(abbr)

    return " ".join(temp2)

def check_punctuation(text):
    data = re.sub("[^a-zA-Z]",' ', text)
    data = re.sub('\[[^]]*\]', ' ', data)
    data = re.sub(r"\\n", " ", data)
    data = data.strip()
    data = ' '.join(data.split())
    return data   

def token_stopwords_lemma(text):
    tokens = word_tokenize(text)
    stop_words2 = ' '.join([word for word in tokens if word not in stop_words])
    data = [lem.lemmatize(word) for word in stop_words2.split()]
    data = ' '.join(data)
    return data

#### Load Model

In [22]:
final_model = tf.keras.models.load_model('gru_model')

2023-04-07 13:27:49.869779: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-04-07 13:27:49.869945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-04-07 13:27:49.870073: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

#### Inference Model

In [11]:
data_inf = ["I'm very unsatisfied with this dress. The material feels cheap, it's falling apart after just one wear, and the sizing is way off. I would not recommend it to anyone.",
"This blouse is a disappointment. The fabric is see-through, the stitching is poor, and the fit is unflattering. I would not buy it again.",
"I absolutely love this dress! The material is high-quality and feels luxurious against my skin. The fit is perfect, and it flatters my figure beautifully. I have received so many compliments every time I wear it. Highly recommend!"]

In [14]:
#convert to dataframe
df_data_inf = pd.DataFrame()
df_data_inf['Review Text'] = data_inf

In [15]:
df_data_inf

,Review Text
0,I'm very unsatisfied with this dress. The mate...
1,This blouse is a disappointment. The fabric is...
2,I absolutely love this dress! The material is ...


#### Cleaning

In [20]:
df_data_inf['Review Text'] = df_data_inf['Review Text'].apply(lambda j: check_chatwords(j))
df_data_inf['Review Text'] = df_data_inf['Review Text'].apply(lambda k: lower(k))
df_data_inf['Review Text'] = df_data_inf['Review Text'].apply(lambda l: check_abbr(l))
df_data_inf['Review Text'] = df_data_inf['Review Text'].apply(lambda m: check_punctuation(m))
df_data_inf['Review Text'] = df_data_inf['Review Text'].apply(lambda n: token_stopwords_lemma(n))

In [21]:
#cleaned data
df_data_inf

,Review Text
0,unsatisfied dress material feel cheap falling ...
1,blouse disappointment fabric see stitching poo...
2,absolutely love dress material high quality fe...


#### Predict

In [23]:
y_pred_inf = final_model.predict(df_data_inf['Review Text'])
y_pred_inf = np.where(y_pred_inf >= 0.5, 1, 0)
y_pred_inf

1/1 [==============================] - 4s 4s/step


array([[0],
       [0],
       [1]])

In [31]:
#combine into dataframe
a = pd.DataFrame()
a['Review Text'] = data_inf
a['Recommended'] = pd.DataFrame(y_pred_inf)
a

,Review Text,Recommended
0,I'm very unsatisfied with this dress. The mate...,0
1,This blouse is a disappointment. The fabric is...,0
2,I absolutely love this dress! The material is ...,1


In [33]:
rec = []
for i in a['Recommended']:
    if i == 0:
        rec.append('No')
    else:
        rec.append('Yes')

a['Recommended_meaning'] = rec
a

,Review Text,Recommended,Recommended_meaning
0,I'm very unsatisfied with this dress. The mate...,0,No
1,This blouse is a disappointment. The fabric is...,0,No
2,I absolutely love this dress! The material is ...,1,Yes


Pada model inference diatas terlihat bahwa untuk row 1 dan 2 terlihat review yang negatif oleh karena itu dikategorikan sebagai kelas 0 / Not recommended dimana customer biasanya tidak akan merekomendasi produk tersebut ke orang lain, sedangkan untuk row 3 terlihat review yang positif sehingga dikategorikan sebagai kelas 1 (Recommended) dimana kelas ini, produk/baju yang dibeli biasanya akan direkomendasi oleh customer. Untuk performa model sudah cukup baik karena sudah bisa mengklasifikasikan review dari customer dengan baik.